# Stacking and back propagation on Mnist with multiple sigmoids

### import data

In [6]:
from sklearn.datasets import fetch_mldata
import numpy as np
from sklearn.utils import shuffle
mnist = fetch_mldata('MNIST original', data_home=".")
mnist.data, mnist.target = shuffle(mnist.data, mnist.target)
X = mnist.data
y_float = mnist.target
y_int = y_float.astype(np.int64)
y = np.eye(10)[y_int]

### Cross Validation

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
rf_train = clf.predict(X_train)
rf_test = clf.predict(X_test)

### add 1 to X

In [9]:
s_train = np.ones(X_train.shape[0])
X_train = np.hstack((X_train, rf_train, s_train.reshape(len(s_train),1)))
s_test = np.ones(X_test.shape[0])
X_test = np.hstack((X_test, rf_test, s_test.reshape(len(s_test),1)))

### PCA

In [10]:
# from sklearn.decomposition import IncrementalPCA

# n_batches = 100
# inc_pca = IncrementalPCA(n_components = 154)
# for X_batch in np.array_split(X_train, n_batches):
#     inc_pca.partial_fit(X_batch)
    
# X_reduced_train = inc_pca.transform(X_train)
# X_reduced_train = np.hstack((X_reduced_train, s_train.reshape(len(s_train),1)))
# X_reduced_test = inc_pca.transform(X_test)
# X_reduced_test = np.hstack((X_reduced_test, s_test.reshape(len(s_test),1)))

### define function

In [11]:
def linear_output_z1(x, w1):
    z1 = np.dot(x, w1)
    return z1

def logistic_output_z2(x, w2):
    z2 = sigmoid(np.dot(x, w2))
    return z2

def sigmoid(a):
    return 1.0 / (1.0 + np.exp(-a))

### define gradient

In [12]:
def logistic_output(x, z1, z2, theta1, theta2, theta3):
    zf = np.matmul(z1, theta1) + np.matmul(z2, theta2) + np.array(np.dot(x, theta3))
    return zf

def gradient_c_theta1(z1, zf, y):
    gradient = np.matmul(np.transpose(z1), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta2(z2, zf, y):
    gradient = np.matmul(np.transpose(z2), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_theta3(x, zf, y):
    gradient = np.matmul(np.transpose(x), (sigmoid(zf)-y)) / N
    return gradient

def gradient_c_w1(x, theta1, zf, y):
    gradient = np.matmul(np.transpose(x), np.transpose(np.matmul(theta1, np.transpose(np.array((sigmoid(zf) - y)))))) / N
    return gradient

def gradient_c_w2(x, w2, theta2, zf, y):
    elemsig = sigmoid(np.matmul(x, w2)) # [5000, 1]
    elem1=np.transpose(np.array(sigmoid(zf) - y)) # [10,5000]
    elem2=np.matmul(theta2, elem1) # [1, 5000]
    elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [1,1]
    elem4 = np.matmul(elemsig, (1 - elem3)) # [5000,1]
    gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

    return gradient


In [13]:
# z1 = linear_output_z1(X_train, int_w1)
# z2 = logistic_output_z2(X_train, int_w2)
# zf = logistic_output(X_train, z1, z2, int_theta1, int_theta2, int_theta3)

# elemsig = sigmoid(np.matmul(X_train, int_w2)) # [52500, 2]
# elem1=np.transpose(np.array(sigmoid(zf) - y_train)) # [10,52500]
# elem2=np.matmul(int_theta2, elem1) # [2, 52500]
# elem3=np.matmul(np.transpose(elemsig), np.transpose(elem2)) # [2,2]
# elem4 = np.matmul(elemsig, (1 - elem3)) # [5000,1]
# # gradient= np.matmul(np.transpose(x), elem4)/ N # [785,1]

### define gradient descent 

In [14]:
def gradient_descent(x, y, int_w1, int_w2, int_theta1, int_theta2, int_theta3, lowtrain, uptrain, iterations):

    place_w1=int_w1
    place_w2=int_w2
    place_theta1=int_theta1
    place_theta2=int_theta2
    place_theta3=int_theta3

    for i in range(iterations):
        trainrate=np.random.uniform(lowtrain,uptrain,1)/np.cbrt(i+1) 
        z1 = linear_output_z1(x, place_w1)
        z2 = logistic_output_z2(x, place_w2)
        zf = logistic_output(x, z1, z2, place_theta1, place_theta2, place_theta3)
        p = sigmoid(zf)

        gradient_theta1 = gradient_c_theta1(z1, zf, y)
        gradient_theta2 = gradient_c_theta2(z2, zf, y)
        gradient_theta3 = gradient_c_theta3(x, zf, y)
        gradient_w1 = gradient_c_w1(x, place_theta1, zf, y)
        gradient_w2 = gradient_c_w2(x, place_w2, place_theta2, zf, y)
        
            
        place_w1 = place_w1 - (trainrate * gradient_w1) - (0.1/N)*place_w1 * trainrate
        place_w2 = place_w2 - (trainrate * gradient_w2) - (0.1/N)*place_w2 * trainrate
        place_theta1 = place_theta1 - trainrate*gradient_theta1*0.01 - (0.1/N)*place_theta1 * trainrate 
        place_theta2 = place_theta2 - trainrate*gradient_theta2 - (0.1/N)*place_theta2 * trainrate
        place_theta3 = place_theta3 - trainrate*gradient_theta3 - (0.1/N)*place_theta3 * trainrate

    
    return  p, place_theta1, place_theta2, place_theta3, place_w1, place_w2


### initialization

In [15]:
N_linear = 1
M_logistic = 4

int_w1, int_w2 = np.random.uniform(-1,1,(X_train.shape[1],N_linear)), np.random.uniform(-1,1,(X_train.shape[1], M_logistic))
int_theta1, int_theta2, int_theta3 = np.random.uniform(-1,1,(N_linear,10)), np.random.uniform(-1,1,(M_logistic,10)),np.random.uniform(-1,1,(X_train.shape[1],10))
result = gradient_descent(X_train, y_train, int_w1, int_w2, int_theta1, int_theta2, int_theta3, 0.00001, 0.001, 1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


In [16]:
def predict_accuracy(xtest, ytest, theta1, theta2, theta3, w1, w2):
    z1 = linear_output_z1(xtest, w1)
    z2 = logistic_output_z2(xtest, w2)
    zf = logistic_output(xtest, z1, z2, theta1, theta2, theta3)
    p = sigmoid(zf)
    prediction = np.argmax(p, axis=1)
    answer = np.argmax(ytest, axis=1)
    success_rate = answer == prediction
    success_rate = success_rate*1
    prediction_rate = sum(success_rate) / xtest.shape[0]
    
    return prediction_rate

In [ ]:
for i in range(500000):
    X_train, y_train = shuffle(X_train, y_train)
    X_train_mini, y_train_mini = X_train[0:500], y_train[0:500]
    N = X_train_mini.shape[0]
    int_w1 = result[4]
    int_w2 = result[5]
    int_theta1 = result[1]
    int_theta2 = result[2]
    int_theta3 = result[3]
    
    rate = 1 / (i + 1)
    rate_100 = rate / 100

    result = gradient_descent(X_train_mini, y_train_mini, int_w1, int_w2, int_theta1, int_theta2, int_theta3, rate_100, rate_100, 10)
    output = predict_accuracy(X_test, y_test, result[1], result[2], result[3], result[4], result[5])
    print(output)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in exp
  # Remove the CWD from sys.path while we load stuff.


0.0998285714286
0.0978857142857
0.0978857142857
0.1688
0.188571428571
0.214
0.208114285714
0.219542857143
0.2056
0.198228571429
0.2012
0.209885714286
0.217028571429
0.204457142857
0.207771428571
0.195314285714
0.192628571429
0.208342857143
0.214342857143
0.203828571429
0.206857142857
0.210171428571
0.2152
0.206057142857
0.21
0.213657142857
0.212514285714
0.201657142857
0.204971428571
0.210628571429
0.213028571429
0.204228571429
0.206628571429
0.201314285714
0.2016
0.206571428571
0.218971428571
0.208457142857
0.209485714286
0.212571428571
0.216342857143
0.216228571429
0.197942857143
0.212685714286
0.205885714286
0.217085714286
0.216628571429
0.220228571429
0.227542857143
0.214
0.217028571429
0.222914285714
0.227771428571
0.217542857143
0.223085714286
0.219257142857
0.222171428571
0.2308
0.226171428571
0.235885714286
0.227485714286
0.226285714286
0.237142857143
0.239942857143
0.238857142857
0.2324
0.230228571429
0.239314285714
0.2396
0.235371428571
0.242971428571
0.234057142857
0.2418857

0.417142857143
0.415257142857
0.413828571429
0.414171428571
0.4136
0.4136
0.411771428571
0.417028571429
0.416971428571
0.4184
0.415371428571
0.417085714286
0.411028571429
0.418514285714
0.415828571429
0.416857142857
0.417257142857
0.4176
0.417714285714
0.418571428571
0.417428571429
0.416057142857
0.415657142857
0.418057142857
0.414685714286
0.414228571429
0.415314285714
0.417771428571
0.416114285714
0.414514285714
0.4188
0.419542857143
0.417714285714
0.416571428571
0.414228571429
0.418057142857
0.414685714286
0.417828571429
0.417885714286
0.417657142857
0.415314285714
0.4192
0.42
0.415428571429
0.414171428571
0.417371428571
0.421371428571
0.4212
0.422742857143
0.4188
0.422228571429
0.421771428571
0.4224
0.4216
0.418228571429
0.421885714286
0.421885714286
0.421771428571
0.420057142857
0.422685714286
0.418514285714
0.418457142857
0.419257142857
0.422628571429
0.422742857143
0.424342857143
0.421828571429
0.420971428571
0.421485714286
0.421028571429
0.422342857143
0.424628571429
0.42251428

0.467714285714
0.465314285714
0.465371428571
0.466571428571
0.468057142857
0.466914285714
0.467942857143
0.467542857143
0.468685714286
0.466857142857
0.469314285714
0.467885714286
0.462057142857
0.466514285714
0.471085714286
0.469028571429
0.470742857143
0.471371428571
0.471371428571
0.470628571429
0.467657142857
0.469257142857
0.469828571429
0.4716
0.471771428571
0.470171428571
0.470171428571
0.465485714286
0.468
0.462285714286
0.468285714286
0.469885714286
0.466057142857
0.469028571429
0.471885714286
0.472342857143
0.471828571429
0.472228571429
0.469257142857
0.469257142857
0.4724
0.472057142857
0.470742857143
0.472228571429
0.470514285714
0.466571428571
0.471828571429
0.472342857143
0.470914285714
0.471657142857
0.474
0.471714285714
0.469771428571
0.472914285714
0.4736
0.467885714286
0.472057142857
0.474114285714
0.473828571429
0.473771428571
0.473314285714
0.474571428571
0.473542857143
0.473657142857
0.473314285714
0.467257142857
0.469257142857
0.4728
0.467142857143
0.468742857143


0.497828571429
0.498685714286
0.499371428571
0.4988
0.498971428571
0.498514285714
0.496571428571
0.4968
0.499485714286
0.499085714286
0.498742857143
0.500171428571
0.500742857143
0.500571428571
0.500114285714
0.4984
0.498057142857
0.499142857143
0.499314285714
0.499828571429
0.497885714286
0.496857142857
0.497885714286
0.497542857143
0.498
0.496628571429
0.497771428571
0.497657142857
0.501314285714
0.501485714286
0.502228571429
0.502285714286
0.501942857143
0.502285714286
0.502057142857
0.501885714286
0.501771428571
0.500228571429
0.501371428571
0.501028571429
0.502057142857
0.502057142857
0.502571428571
0.501942857143
0.5024
0.502857142857
0.5028
0.502971428571
0.501885714286
0.500914285714
0.497885714286
0.498285714286
0.499028571429
0.500342857143
0.5016
0.5016
0.501314285714
0.501657142857
0.499028571429
0.499942857143
0.498742857143
0.500571428571
0.500628571429
0.497314285714
0.498114285714
0.502057142857
0.501257142857
0.500742857143
0.499542857143
0.500285714286
0.502057142857


...

0.599314285714
0.599257142857
0.599428571429
0.599142857143
0.5992
0.599657142857
0.599542857143
0.599428571429
0.598857142857
0.599028571429
0.599542857143
0.599542857143
0.599371428571
0.599371428571
0.600285714286
0.600685714286
0.600228571429
0.600114285714
0.599828571429
0.599885714286
0.599657142857
0.600285714286
0.601428571429
0.599714285714
0.5992
0.599314285714
0.600057142857
0.6004
0.6004
0.6
0.5996
0.599771428571
0.600228571429
0.599657142857
0.599542857143
0.599371428571
0.599142857143
0.599371428571
0.599657142857
0.6008
0.600971428571
0.601142857143
0.600228571429
0.601085714286
0.600914285714
0.600857142857
0.6008
0.6004
0.600628571429
0.600457142857
0.6004
0.599371428571
0.599485714286
0.599942857143
0.599428571429
0.599371428571
0.6
0.6004
0.601085714286
0.600571428571
0.600457142857
0.599828571429
0.600628571429
0.601371428571
0.6008
0.600514285714
0.599542857143
0.599485714286
0.599942857143
0.599828571429
0.598971428571
0.5996
0.599485714286
0.599485714286
0.5996
0

0.603942857143
0.603885714286
0.603885714286
0.604171428571
0.604342857143
0.604285714286
0.604342857143
0.604
0.6044
0.6048
0.604514285714
0.604171428571
0.604057142857
0.604971428571
0.604514285714
0.604457142857
0.604114285714
0.604457142857
0.604114285714
0.604171428571
0.6044
0.604114285714
0.603942857143
0.604285714286
0.604228571429
0.604742857143
0.604571428571
0.604114285714
0.604285714286
0.6044
0.604
0.604171428571
0.604228571429
0.604285714286
0.604457142857
0.604171428571
0.604742857143
0.604457142857
0.604171428571
0.604971428571
0.605142857143
0.605371428571
0.605085714286
0.605314285714
0.605485714286
0.606
0.605142857143
0.605085714286
0.6052
0.605142857143
0.604228571429
0.6044
0.604914285714
0.605257142857
0.605085714286
0.6048
0.605257142857
0.604971428571
0.605257142857
0.6056
0.6056
0.605485714286
0.605485714286
0.605485714286
0.605371428571
0.6056
0.605714285714
0.605828571429
0.605542857143
0.605371428571
0.605542857143
0.605085714286
0.605371428571
0.6052571428

0.611142857143
0.610857142857
0.611428571429
0.610914285714
0.611371428571
0.610971428571
0.611257142857
0.611257142857
0.611542857143
0.611771428571
0.611828571429
0.611942857143
0.612228571429
0.612171428571
0.612114285714
0.611771428571
0.611942857143
0.611885714286
0.611257142857
0.61
0.610171428571
0.610228571429
0.610742857143
0.610571428571
0.610685714286
0.610571428571
0.610742857143
0.6108
0.611542857143
0.6116
0.611771428571
0.611542857143
0.611257142857
0.610742857143
0.611028571429
0.610628571429
0.610742857143
0.611028571429
0.6108
0.611142857143
0.611142857143
0.611257142857
0.611257142857
0.611257142857
0.611142857143
0.610514285714
0.610171428571
0.611314285714
0.611142857143
0.6112
0.611314285714
0.611314285714
0.611371428571
0.611371428571
0.611257142857
0.611485714286
0.611428571429
0.611771428571
0.611714285714
0.611771428571
0.611657142857
0.611942857143
0.611885714286
0.612171428571
0.611828571429
0.612
0.6116
0.611542857143
0.6116
0.611714285714
0.611485714286
0.

0.614457142857
0.615142857143
0.615257142857
0.615085714286
0.615085714286
0.615028571429
0.615085714286
0.6152
0.615428571429
0.615485714286
0.615257142857
0.615542857143
0.615428571429
0.615657142857
0.6156
0.615828571429
0.615828571429
0.615485714286
0.6152
0.6156
0.614742857143
0.6156
0.615828571429
0.615542857143
0.615657142857
0.6156
0.615828571429
0.615714285714
0.615828571429
0.615828571429
0.615885714286
0.615942857143
0.615885714286
0.616457142857
0.616171428571
0.616914285714
0.616342857143
0.6164
0.616285714286
0.616057142857
0.616114285714
0.616285714286
0.615828571429
0.616228571429
0.616285714286
0.616114285714
0.616171428571
0.616228571429
0.615657142857
0.616285714286
0.616171428571
0.615828571429
0.616457142857
0.6172
0.617314285714
0.616457142857
0.616114285714
0.6164
0.616285714286
0.6164
0.616914285714
0.616628571429
0.616285714286
0.6164
0.616342857143
0.616285714286
0.616342857143
0.616514285714
0.616628571429
0.616914285714
0.615828571429
0.6156
0.615828571429
0

0.618914285714
0.619942857143
0.619885714286
0.619371428571
0.618514285714
0.618457142857
0.619714285714
0.619371428571
0.619428571429
0.619085714286
0.619714285714
0.618914285714
0.619542857143
0.620228571429
0.619428571429
0.619771428571
0.619657142857
0.620457142857
0.620514285714
0.620571428571
0.6212
0.621542857143
0.621314285714
0.6212
0.621028571429
0.620914285714
0.620914285714
0.620971428571
0.620742857143
0.621028571429
0.621028571429
0.621085714286
0.621085714286
0.620857142857
0.620742857143
0.620857142857
0.620914285714
0.620971428571
0.620857142857
0.620971428571
0.620571428571
0.620857142857
0.6204
0.620971428571
0.621085714286
0.621028571429
0.621028571429
0.620742857143
0.6208
0.6208
0.621028571429
0.620914285714
0.620914285714
0.620971428571
0.6208
0.620971428571
0.620914285714
0.620914285714
0.620914285714
0.620914285714
0.6208
0.620571428571
0.620628571429
0.620457142857
0.620914285714
0.620971428571
0.621028571429
0.621028571429
0.620114285714
0.620342857143
0.6200

### save array

In [ ]:
nparray = np.array(result)
np.save('result.npy', nparray)

In [ ]:
rate = 1 / (5 + 1)

In [ ]:
rate / 100

In [ ]:
    rate = 1 / (i + 1)